In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import json
import re
import warnings
warnings.filterwarnings('ignore')
#import time
import os

In [3]:
def get_soup(url):
    html_content = requests.get(url).text
    soup = BeautifulSoup(html_content, "html.parser")
    return soup

In [12]:
df = pd.DataFrame()
for year in range(2014, 2020, 1): # starts from 2014 and ends at 2019    
    for start in range(1, 600, 50):
        print(year, start)
        URL = f"https://www.imdb.com/search/title/?title_type=feature&year={year}-01-01,{year}-12-31&start={start}&ref_=adv_nxt"
        
        # GET ALL THE FILM DETAILS (Name, ID and links)
        film_details = get_soup(URL).findAll('h3')
        
        for i, film_detail in enumerate(film_details):
            if i == 50:
                break
            href_link = film_detail.find('a')
            film_name = href_link.text              # Movie Name
            film_id = str(href_link).split('/')[2]  # Movie ID
            
            # Let us go to link in each movie and get all the cast details
            link = f"https://www.imdb.com/title/{film_id}/fullcredits?ref_=tt_ov_st_sm"
            new_soup = get_soup(link)
            table = new_soup.findAll('table', class_='cast_list')
            
            names, nameIDs, char_nameIDs, titles, general_titles = [],[],[],[],[]
            
            # We can modify this code for any idx
            categories = ['Director', 'Writer', 'Producer', 'Composer', 'Cinematographer', 'Movie Editor', 'Casted by', 
                          'Production Designer', 'Art Director', 'Set Decorator', 'Costume Designer', 'Makeup Artist', 
                          'Production Manager','Second Unit_Assistant Director', 'Art dept', 'Sound dept', 'Special effects', 
                          'Visual effects', 'Stunts', 'Camera and Electrical dept', 'Casting dept','Costune and Wardrobe dept',
                          'Editorial dept','Location management','Music dept','Script and continuity dept',
                          'Transportation dept', 'Additional crew', 'Thanks']
            '''
            We just need info on director, writer, producer and cast details so we would just retrive those indexes
            Director = 0, Writer = 1, Producer = 3
            '''
            directors_cnt, writers_cnt, producers_cnt = 0, 0, 0
            for idx, table in enumerate(new_soup.findAll('table', class_='simpleTable simpleCreditsTable')):
                if idx in [0, 1, 2]:
                    # Get td class=name and class=credits
                    all_ids_and_names = table.findAll('a')
                    
                    if idx == 0: # No directors sometimes !
                        if 'Directed by' in str(new_soup):
                            #print('Yes D')
                            for id_name in all_ids_and_names:
                                names.append(id_name.text.strip())
                                nameIDs.append(str(id_name).split('/')[2])
                            directors_cnt = len(names)
                            titles += ['directed by'] * directors_cnt
                            general_titles += ['NONE'] * directors_cnt  
                    if idx == 1: # No writers sometimes
                        if 'Writing Credits' in str(new_soup):
                            #print('Yes W')
                            for id_name in all_ids_and_names:
                                names.append(id_name.text.strip())
                                nameIDs.append(str(id_name).split('/')[2])
                            writers_cnt = len(names) - directors_cnt
                            titles += ['written by'] * writers_cnt
                            general_titles += ['NONE'] * writers_cnt
                    if idx == 2: # No producers sometimes
                        if 'Produced by' in str(new_soup):
                            #print('Yes P')
                            for id_name in all_ids_and_names:
                                names.append(id_name.text.strip())
                                nameIDs.append(str(id_name).split('/')[2])
                        
                            all_credits = table.findAll('td', {'class':'credit'})
                            for credit in all_credits:
                                titles.append(credit.text.strip())
                            general_title = categories[idx]
                            producers_cnt = len(names) - directors_cnt - writers_cnt
                            general_titles += [general_title] * producers_cnt

            
            cast_nameIDs, cast_titles, cast_names = [],[],[]
            # Getting all cast details (both the main cast and other cast actors)
            for table in new_soup.findAll('table', class_='cast_list'):
                all_casts = table.findAll('a')
                
                # Add the names of the cast/actors
                for idx, cast in enumerate(all_casts):
                    if '/name/' in str(cast) and 'img alt' not in str(cast):
                        nameID = str(cast).split('/')[2]
                        cast_name = str(cast).split('/')[-2].replace('>','').replace('<','').replace('"','').strip()
                        cast_nameIDs.append(nameID)
                        cast_names.append(cast_name)
                        
            cast_titles = ['Actor - Cast'] * len(cast_names)
            
            nameIDs += cast_nameIDs
            titles  += cast_titles
            names   += cast_names
            general_titles += cast_titles
                    
            new_df = pd.DataFrame()
            
            try:
                new_df['Names']         = names
                new_df['Popularity']    = i + 1
                new_df['IDs']           = nameIDs
                new_df['General Title'] = general_titles
                new_df['Specific role'] = titles
                new_df['movieTitle']    = film_name
                new_df['movieID']       = film_id
                new_df['Year']          = year
            except:
                new_df = new_df[new_df['Popularity']!= i+1]
            df = df.append(new_df)
    df.to_excel(f"imdb_first_600_{year}.xlsx", index=False)

2016 1
2016 51
2016 101
2016 151
2016 201
2016 251
2016 301
2016 351
2016 401
2016 451
2016 501
2016 551
